<a href="https://colab.research.google.com/github/ML-Bioinfo-CEITEC/cDNA-pretraining/blob/main/experiments/kmer_tokenization/SentencePieceTokenizer_for_DNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq transformers datasets --quiet

In [2]:
# #CONFIG
limit = 263659 #263659 is full dataset
vocab_size_limit = 30000 #30000
hf_name = 'Vlasta'

In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

In [4]:
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Split
from tokenizers.processors import BertProcessing
from tokenizers import Regex
from tokenizers import SentencePieceBPETokenizer, SentencePieceUnigramTokenizer

In [5]:
ds = load_dataset("simecek/Human_DNA_v0")
train_portion = ds['train']['Seq']

Using custom data configuration simecek--Human_DNA_v0-d7be3fc44fadbb72
Reusing dataset parquet (/root/.cache/huggingface/datasets/simecek___parquet/simecek--Human_DNA_v0-d7be3fc44fadbb72/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
len(train_portion)

263659

In [7]:
def batch_iterator(dataset, batch_size):
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]

In [ ]:
%%time
#TODO try to split dataset sequence lengths
tokenizer_name = f"DNA_SentencepieceBPE_max_vocab_{vocab_size_limit}"

unk_token = "[UNK]"  
spl_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] 

tokenizer = SentencePieceBPETokenizer(unk_token=unk_token)

tokenizer.post_processor = BertProcessing(sep=('[SEP]', spl_tokens.index('[SEP]')), cls=('[CLS]', spl_tokens.index('[CLS]')))
TRA = train_portion[:limit]

#Default vocab_size_limit is 30000
tokenizer.train_from_iterator(    
    TRA,
    vocab_size=vocab_size_limit,
    special_tokens=spl_tokens,
) 

# tokenizer.train_from_iterator(
#     batch_iterator(TRA, batch_size=1000),
#     vocab_size=vocab_size_limit,
#     special_tokens=spl_tokens
# ) 

print(f"BPE tokenizer vocab size: {tokenizer.get_vocab_size()}")



In [ ]:
tokens = sorted(tokenizer.get_vocab().keys(), key=len)
print('SHORTEST TOKENS\n', *tokens[:10])
print(f'\nLONGEST TOKENS (max {len(tokens[-1])})', *tokens[-10:], sep='\n')
tokenizer_name = tokenizer_name+f'_max_tokenlen_{len(tokens[-1])}'

In [ ]:
encoding = tokenizer.encode('NNACTGACACGAAAAAAAGGGC[MASK]GCGCAACTCCAG')
print(encoding.ids)
print(encoding.tokens)
print(tokenizer.decode(encoding.ids))

In [ ]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    model_max_length = 512,
    padding_side='right',
    truncation_side='right',
    pad_token='[PAD]',
    mask_token='[MASK]',
    sep_token='[SEP]',
    cls_token='[CLS]',
    unk_token='[UNK]',
  )

fast_tokenizer.push_to_hub(tokenizer_name)

In [ ]:
from transformers import AutoTokenizer
myDownloadedTokenizer = AutoTokenizer.from_pretrained(f"{hf_name}/{tokenizer_name}")
# myDownloadedTokenizer = fast_tokenizer
dnabert_tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")

long_sequence = '[MASK]' + 'AAAAAA '*1000 
short_sequence = 'NNACTGACACGAAAAAAAGGGC[MASK]GCGCAACTCCAG'
long_encoded_dnabert = dnabert_tokenizer(long_sequence)
long_encoded_ours = myDownloadedTokenizer(long_sequence)
#Padding doesnt show up for
short_encoded_dnabert = dnabert_tokenizer(short_sequence, padding=True)
short_encoded_ours = myDownloadedTokenizer(short_sequence, padding=True)
print("\nDNABERT BEHAVIOR")
print(long_encoded_dnabert)
print(dnabert_tokenizer.decode(long_encoded_dnabert['input_ids']))
print(short_encoded_dnabert)
print(dnabert_tokenizer.decode(short_encoded_dnabert['input_ids']))
print(dnabert_tokenizer.special_tokens_map)
print(dnabert_tokenizer.padding_side)
print('\nOURS BEHAVIOR')
print(long_encoded_ours)
print(myDownloadedTokenizer.decode(long_encoded_ours['input_ids']))
print(short_encoded_ours)
print(myDownloadedTokenizer.decode(short_encoded_ours['input_ids']))
print(myDownloadedTokenizer.special_tokens_map)
print(myDownloadedTokenizer.padding_side)



